# Predict Trained Model on Actual Test Data 

In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
import joblib
from sklearn.preprocessing import Normalizer
%load_ext autoreload
%autoreload 2

Load the saved model

In [2]:
rf_mod = joblib.load("../models/sp_model_final.joblib")
rf_mod

RandomForestClassifier(class_weight={0: 0.84, 1: 0.17}, max_depth=6,
                       max_features=2, min_samples_leaf=8, n_estimators=125,
                       random_state=8)

All good.

Load the Test data from csv file

In [3]:
df=pd.read_csv('../data/raw/test.csv')

Performing basic investigation into the data 

In [4]:
df.head(5)

,Id_old,Id,GP,MIN,PTS,FGM,FGA,FG%,3P Made,3PA,...,FTM,FTA,FT%,OREB,DREB,REB,AST,STL,BLK,TOV
0,1,0,56,9.1,4.0,1.6,3.7,43.7,0.1,0.3,...,0.7,1.2,63.4,1.2,0.8,1.7,0.4,0.2,0.3,0.8
1,8194,1,43,19.3,10.1,3.7,8.1,46.0,0.6,1.7,...,1.8,2.5,75.3,0.5,0.9,1.5,3.5,0.6,0.0,1.8
2,3,2,82,33.9,11.3,4.9,10.6,45.6,0.5,1.9,...,1.8,2.7,71.2,1.3,3.3,4.5,2.5,1.3,0.3,2.0
3,8196,3,86,44.7,18.8,6.8,15.9,42.9,0.5,1.8,...,4.5,6.3,70.9,1.5,3.2,5.0,4.1,0.9,0.1,3.6
4,8197,4,58,12.3,4.7,1.6,4.0,40.0,0.5,1.7,...,1.1,1.3,76.9,0.2,0.6,0.9,1.5,0.5,-0.4,0.9


In [5]:
df.shape

(3799, 21)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3799 entries, 0 to 3798
Data columns (total 21 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   Id_old   3799 non-null   int64  
 1   Id       3799 non-null   int64  
 2   GP       3799 non-null   int64  
 3   MIN      3799 non-null   float64
 4   PTS      3799 non-null   float64
 5   FGM      3799 non-null   float64
 6   FGA      3799 non-null   float64
 7   FG%      3799 non-null   float64
 8   3P Made  3799 non-null   float64
 9   3PA      3799 non-null   float64
 10  3P%      3799 non-null   float64
 11  FTM      3799 non-null   float64
 12  FTA      3799 non-null   float64
 13  FT%      3799 non-null   float64
 14  OREB     3799 non-null   float64
 15  DREB     3799 non-null   float64
 16  REB      3799 non-null   float64
 17  AST      3799 non-null   float64
 18  STL      3799 non-null   float64
 19  BLK      3799 non-null   float64
 20  TOV      3799 non-null   float64
dtypes: float64(18)

In [7]:
df.describe()

,Id_old,Id,GP,MIN,PTS,FGM,FGA,FG%,3P Made,3PA,...,FTM,FTA,FT%,OREB,DREB,REB,AST,STL,BLK,TOV
count,3799.000000,3799.000000,3799.000000,3799.000000,3799.000000,3799.000000,3799.000000,3799.000000,3799.000000,3799.000000,...,3799.000000,3799.000000,3799.000000,3799.000000,3799.000000,3799.000000,3799.000000,3799.000000,3799.000000,3799.000000
mean,7010.614109,1899.000000,62.853909,18.650224,7.328034,2.835404,6.302580,44.599079,0.255962,0.796920,...,1.399842,1.953567,71.612924,1.096025,2.179495,3.275783,1.636483,0.653593,0.257726,1.257910
std,3954.173641,1096.821164,17.151740,8.727259,4.294724,1.688427,3.579221,6.040168,0.380987,1.052862,...,0.926140,1.250376,10.457336,0.785678,1.371935,2.070646,1.335496,0.410573,0.639660,0.712449
min,1.000000,0.000000,6.000000,3.700000,0.700000,0.300000,0.800000,25.100000,-1.000000,-2.700000,...,0.000000,0.000000,23.700000,0.000000,0.200000,0.300000,0.000000,0.000000,-7.100000,0.100000
25%,3644.000000,949.500000,51.000000,12.200000,4.200000,1.600000,3.700000,40.500000,0.000000,0.100000,...,0.700000,1.000000,65.000000,0.500000,1.200000,1.800000,0.600000,0.400000,0.100000,0.700000
50%,7062.000000,1899.000000,63.000000,17.000000,6.400000,2.500000,5.500000,44.600000,0.300000,0.800000,...,1.200000,1.700000,71.500000,0.900000,1.900000,2.800000,1.300000,0.600000,0.200000,1.100000
75%,10402.500000,2848.500000,74.000000,23.300000,9.400000,3.700000,8.100000,48.500000,0.500000,1.500000,...,1.900000,2.600000,78.000000,1.500000,2.900000,4.300000,2.300000,0.900000,0.400000,1.600000
max,13792.000000,3798.000000,126.000000,68.000000,33.000000,13.400000,26.200000,74.600000,1.600000,4.300000,...,7.800000,9.800000,127.100000,6.900000,12.000000,18.500000,9.000000,2.700000,14.800000,5.200000


Dropping feature columns not used in model traning

In [8]:
df_cleaned = df.drop(['Id_old','Id', '3P Made', '3PA','3P%', 'BLK'], axis=1)

In [9]:
df_cleaned.shape

(3799, 15)

In [11]:
x=df_cleaned.copy()

Applying the same Feature Scaling 

In [12]:
x = Normalizer().fit_transform(x) 
x

array([[0.58421991, 0.09493574, 0.04172999, ..., 0.004173  , 0.0020865 ,
        0.008346  ],
       [0.42539784, 0.19093438, 0.09991903, ..., 0.03462541, 0.00593578,
        0.01780735],
       [0.66203627, 0.27369548, 0.09123183, ..., 0.02018403, 0.0104957 ,
        0.01614723],
       ...,
       [0.56298842, 0.10516199, 0.02230709, ..., 0.00424897, 0.00424897,
        0.00531121],
       [0.63169009, 0.27183967, 0.1029158 , ..., 0.02626127, 0.00922693,
        0.01703434],
       [0.52746299, 0.11508283, 0.02877071, ..., 0.00575414, 0.00287707,
        0.01150828]])

#### Predict Result and Export to CSV for uploading to Kaggle

In [14]:
y_pred=rf_mod.predict(x)
y_pred_proba=rf_mod.predict_proba(x)

In [15]:
np.unique(y_pred,return_counts=True)

(array([0, 1], dtype=int64), array([1179, 2620], dtype=int64))

In [16]:
y_pred_proba

array([[0.46268963, 0.53731037],
       [0.61192428, 0.38807572],
       [0.20632945, 0.79367055],
       ...,
       [0.55070382, 0.44929618],
       [0.36325161, 0.63674839],
       [0.5511656 , 0.4488344 ]])

In [20]:
rf_mod.classes_

array([0, 1], dtype=int64)

In [21]:
df['TARGET_5Yrs'] = y_pred_proba[:,1]

In [22]:
df.to_csv('../data/processed/TestResult_RF_Final.csv',index=False,columns=['Id', 'TARGET_5Yrs'])